# Setup

In [33]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn import linear_model
import xgboost as xgb


def binary_treatment(treatment_code, treatment):    
    """
    less than a year vs. more than a year
    """
    if treatment_code==1:
        return (treatment > 0).astype("int")
    """
    less than two yeas vs. more than two years
    """
    if treatment_code==2:
        return  (treatment > 1).astype('int')
    
    """
    less than a year vs. more than a year but less than two
    """
    if treatment_code==3:
        return treatment[(treatment==0) | (treatment==1)]
    
    """
    1years<=Duration<2years vs. 2years<=Duration
    """
    if treatment_code==4:
        relevant = treatment[(treatment > 0)]
        return (relevant >=2).astype("int")
    
    """
    2years<=Duration<3years vs. 3years<=Duration<4years 
    """
    if treatment_code==5:
        relevant = treatment[(treatment==2) | (treatment==3)]
        return (relevant == 3).astype("int")
    

def data_preprocessing(country_data, treatment_code):
    data = country_data.drop(columns=['CNTRYID', 'CNT', 'CNTSTUID', 
                                      'preprimary_start_theo', 'primary_start_theo']).copy()
    treatment = binary_treatment(treatment_code, data['DURECEC'])
    data = data.loc[treatment.index, :]
    outcome = data['PV1READ']
    categorial_properties = ['ST003D02T','ST003D03T', 'STRATUM','LANGTEST_QQQ', 'IMMIG', 'ST001D01T',
                             'ST004D01T', 'ST013Q01TA', 'MISCED', 'FISCED',
                             'ST126Q01TA','ST022Q01TA', 'ST023Q01TA', 'ST023Q02TA', "OCOD1", "OCOD2", 
                            ]
    numerical_properties = ["ST021Q01TA", "BMMJ1", "BFMJ2", "ESCS", "WEALTH"]
    
    numerical_covariates = data[numerical_properties]
    categorial_covariates = data[categorial_properties]
    
    # scaling numerical properties
    scaler = preprocessing.StandardScaler()
    scaler.fit(numerical_covariates)
    x_scaled = scaler.transform(numerical_covariates)
    
    # encoding categorial covariates
    ohe = preprocessing.OneHotEncoder()
    ohe.fit(categorial_covariates)
    categorial_encoding = ohe.transform(categorial_covariates).toarray()
    
    #remove dependent categorial columns
    variables_in_category = [data[col].nunique() for col in categorial_properties]
    columns_to_remove = [variables_in_category[0]-1]
    for i in range(1, len(variables_in_category)):
        columns_to_remove.append(variables_in_category[i] + columns_to_remove[i-1])
    categorial_encoding = np.delete(categorial_encoding, columns_to_remove, 1)
    
    X = np.concatenate([x_scaled, categorial_encoding], axis=1)

    return X, treatment, outcome


def estimate_propensity(X, treatment):
    # Learn propensity score
    lr_learner = LogisticRegression(C=10, solver='lbfgs', max_iter=10000)
    lr_learner.fit(X, treatment)
    propensity_score = lr_learner.predict_proba(X)[:, 1]   
    return propensity_score


def trim_common_support(X, treated_propensity_score, control_propensity_score, propensity_scores, treatment, outcome):
    """
    Trim data that does not appear to maintain common support, using min max approach on propensity core
    """
    min_treated = np.min(treated_propensity_score)
    max_treated = np.max(treated_propensity_score)
    min_control = np.min(control_propensity_score)
    max_control = np.max(control_propensity_score)
    max_min = np.maximum(min_control, min_treated)
    min_max = np.minimum(max_control, max_treated)

    indices_smaller_than_max_min = np.argwhere(propensity_scores < max_min)
    indices_greater_than_min_max = np.argwhere(propensity_scores > min_max)
    rows_to_delete = np.concatenate([indices_greater_than_min_max, indices_smaller_than_max_min])
    rows_to_delete = rows_to_delete.reshape((-1,))
    exclude_idx = set(rows_to_delete)
    mask = np.array([(i in exclude_idx) for i in range(len(X))])

    return X[~mask], propensity_scores[~mask], treatment[~mask], outcome[~mask]

In [37]:
data = pd.read_csv("PISA2018_data.csv")
data = data[data["DURECEC"]<7].copy()

In [38]:
# uncommend the desired country
country = data[(data["CNT"] == "CAN") & (data["LANGTEST_QQQ"]==313)].copy()
# country = data[(data["CNT"] == "AUS")].copy()
# country = data[(data["CNT"] == "IRL") & (data["LANGTEST_QQQ"]==313)].copy()

In [39]:
# preprocess and trim the data according to the desired treatment
X, treatment, outcome = data_preprocessing(country, treatment_code=1)
propensity = estimate_propensity(X, treatment)
treated_propensity_score = propensity[treatment == 1]
control_propensity_score = propensity[treatment == 0]
X, propensity, treatment, outcome = trim_common_support(X, treated_propensity_score, control_propensity_score, propensity, treatment, outcome)

In [36]:
# generating samples as csv file for further analysis
covariates = pd.DataFrame(X)
treatment_outcome = pd.DataFrame({"T":treatment, "outcome":outcome})
pd.concat([covariates, treatment_outcome.reset_index(drop=True)], 
          axis=1).to_csv("data_by_country/canada_treatment_4.csv", index=False)